# 이미지 요약 RAG

### LLM 초기화

* 이미지에서 설명을 추출할 llava와 챗봇용 llama 3.1 모델을 선언함.

In [1]:
from langchain_community.llms import Ollama

# 이미지 설명용 모델 (LLaVA)
vlm = Ollama(model="llava:7b")

# QA용 모델 (LLaMa)
llm = Ollama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0
)

C:\Users\IISLab\AppData\Local\Temp\ipykernel_33664\802893297.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  vlm = Ollama(model="llava:7b")


### 이미지를 설명하는 문장을 추출하는 함수

* 이미지와 텍스트로 된 프롬프트를 LlaVa에 입력하여 이미지에 대한 설명을 추출함.

In [2]:
import base64

# 이미지를 입력으로 받아 llava가 이미지의 내용을 추출 
def describe_image(image_path: str) -> str:
    with open(image_path, "rb") as img_file:
        img_bytes = img_file.read()
    base64_img = base64.b64encode(img_bytes).decode("utf-8")
    image_summary_prompt = """
    You are an AI assistant specialized in summarizing visual content.
    Please look at the provided image and describe the most important details in a clear and concise manner.
    Do not make assumptions or add imagined details—only describe what is visible.
    Keep the summary to 2–4 sentences."""
    result = vlm.invoke(image_summary_prompt, images=[base64_img])
    return result.strip()

### Document 생성

* 각 이미지 설명과 이미지 경로를 Document 객체로 만들어서 리스트에 저장함.

In [3]:
from langchain_core.documents import Document
import os

IMAGE_FOLDER = "./images/"
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(('jpg', 'jpeg', 'png'))]

# 각 이미지의 설명을 Document에 저장
docs = []
for idx, img_name in enumerate(image_files):
    img_path = os.path.join(IMAGE_FOLDER, img_name)
    description = describe_image(img_path)
    doc = Document(page_content=description, metadata={"source": {img_name}})
    docs.append(doc)

### Vector Store 생성

* Document를 텍스트 임베딩을 이용해 벡터로 만들고 DB에 저장함.

In [4]:
from langchain_community.embeddings import OllamaEmbeddings

# 텍스트 임베딩 생성기
embedding = OllamaEmbeddings(model="daynice/kure-v1")

C:\Users\IISLab\AppData\Local\Temp\ipykernel_33664\3134017166.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="daynice/kure-v1")


In [5]:
from langchain_community.vectorstores import FAISS

# 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=docs, embedding=embedding)
# 벡터 스토어 저장
vectorstore.save_local("./Image_description")
# retriever 생성
retriever = vectorstore.as_retriever()

### 프롬프트 초기화

##### QA용 프롬프트
* 사용자의 질문(question)과 검색된 이미지 요약(context)를 입력으로 받음

In [6]:
from langchain_core.prompts import PromptTemplate

# 이미지 설명 기반 QA를 위한 프롬프트 템플릿
chat_prompt = PromptTemplate.from_template(
"""당신은 이미지 설명(Image Description)에 기반한 질문-답변(Question-Answering)을 수행하는 AI 어시스턴트입니다. 
당신의 임무는 제공된 이미지에 대한 설명(context)을 바탕으로 사용자의 질문(question)에 답변하는 것입니다.

아래의 이미지 설명(context)을 참고하여 질문(question)에 답하세요. 
만약 설명 안에 답이 없거나 답을 유추하기 어렵다면, `주어진 이미지 설명에서는 해당 질문에 대한 답을 찾을 수 없습니다` 라고 답하세요. 

#Question:
{question}

#Image Description (Context):
{context}

#Answer:"""
)

In [7]:
from langchain_core.output_parsers import StrOutputParser

# 체인을 생성합니다.
chat_chain = chat_prompt | llm | StrOutputParser()

### DB 검색 및 질문과 관련된 이미지 요약

* 영어로 DB에서 유사한 Document를 검색하여 이미지 설명을 찾고, QA용 챗봇 체인에 입력하여 이미지 설명을 요약

##### 검색된 이미지들의 설명을 요약한 결과

In [8]:
question = "사람이 직접 냉장고를 사용하고 있는 모습이 포함된 이미지를 요약해주세요."
# 벡터 DB 에서 참고할 문서 검색
retrieved_docs = retriever.invoke(question)
print(f"retrieved size: {len(retrieved_docs)}")
combined_docs = "\n\n".join(doc.page_content for doc in retrieved_docs)
# 검색된 문서를 첨부해서 PROMPT 생성
formatted_prompt = {"context": combined_docs, "question": question}
# 체인을 실행하고 결과를 stream 형태로 출력
result = ""
for chunk in chat_chain.stream(formatted_prompt):
    print(chunk, end="", flush=True)
    result += chunk
print()

retrieved size: 4
이미지 요약: 냉장고 앞에서 사람과 냉장고가 함께 있는 이미지를 보여줍니다. 냉장고는 다양한 기능을 제공하는 디지털 스크린을 가지고 있으며, 이는 현대적인 주방 기기의 특징입니다.


##### 검색된 이미지들의 파일명

In [9]:
for doc in retrieved_docs:
    print(doc.metadata['source'])

{'refri7.png'}
{'refri9.png'}
{'refri4.png'}
{'refri2.png'}


ppt 제작 라이브러리 다운로드

In [10]:
!pip install python-pptx


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN
import os

def create_simple_ppt(retrieved_docs, result_text, output_filename):
    
    prs = Presentation()
    
    # 각 문서의 source(파일명)으로 이미지 찾아서 슬라이드 생성
    for i, doc in enumerate(retrieved_docs, 1):
        # 슬라이드 추가
        slide_layout = prs.slide_layouts[6]  # 빈 슬라이드
        slide = prs.slides.add_slide(slide_layout)
        
        # 파일명에서 이미지 경로 추출 (set 형태)
        source_data = doc.metadata['source']
        image_path = list(source_data)[0]  # set에서 첫 번째 값
        
        # 이미지 폴더 경로와 합치기
        full_image_path = os.path.join("./images/", image_path)
        
        # 이미지가 존재하면 추가
        if os.path.exists(full_image_path):
            # 슬라이드 크기 (표준 16:9)
            slide_width = Inches(10)  # 슬라이드 너비
            slide_height = Inches(7.5)  # 슬라이드 높이
            
            # 이미지 영역을 위한 공간 (하단에 텍스트 공간 확보)
            max_width = Inches(9)   # 좌우 여백 0.5인치씩
            max_height = Inches(5.5)  # 상하 여백 + 하단 텍스트 공간 확보
            
            # 이미지를 최대 크기에 맞춰 비율 유지하며 추가
            left = Inches(0.5)  # 좌측 여백
            top = Inches(0.5)   # 상단 여백
            
            # 이미지 추가 (자동으로 비율 유지하면서 최대 크기 제한)
            pic = slide.shapes.add_picture(full_image_path, left, top, width=max_width)
            
            # 높이가 넘어가면 높이 기준으로 다시 조정
            if pic.height > max_height:
                # 높이 기준으로 비율 맞춰서 다시 설정
                slide.shapes._spTree.remove(pic._element)  # 기존 이미지 제거
                pic = slide.shapes.add_picture(full_image_path, left, top, height=max_height)
                
                # 가로가 넘어가면 중앙 정렬
                if pic.width <= max_width:
                    pic.left = int((slide_width - pic.width) / 2)
            else:
                # 가로 중앙 정렬
                pic.left = int((slide_width - pic.width) / 2)
            
            # 이미지 파일명을 하단에 텍스트박스로 추가
            textbox = slide.shapes.add_textbox(
                left=Inches(0.5),
                top=Inches(6.5),  # 슬라이드 하단
                width=Inches(9),
                height=Inches(0.8)
            )
            
            text_frame = textbox.text_frame
            text_frame.text = os.path.basename(image_path)  # 파일명만 추출
            
            # 텍스트 스타일 설정
            paragraph = text_frame.paragraphs[0]
            paragraph.alignment = PP_ALIGN.CENTER  # 중앙 정렬
            font = paragraph.font
            font.size = Inches(0.2)  # 폰트 크기
            font.bold = True
            
            print(f"이미지 추가됨: {full_image_path}")
        else:
            # 이미지가 없는 경우 파일명만 표시
            textbox = slide.shapes.add_textbox(
                left=Inches(1),
                top=Inches(3),
                width=Inches(8),
                height=Inches(1.5)
            )
            
            text_frame = textbox.text_frame
            text_frame.text = f"이미지를 찾을 수 없습니다:\n{image_path}"
            
            # 텍스트 스타일 설정
            paragraph = text_frame.paragraphs[0]
            paragraph.alignment = PP_ALIGN.CENTER
            font = paragraph.font
            font.size = Inches(0.25)
            
            print(f"이미지 파일을 찾을 수 없음: {full_image_path}")
    
    # 마지막 슬라이드: 요약 페이지
    slide_layout = prs.slide_layouts[1]  # 제목+내용 레이아웃
    slide = prs.slides.add_slide(slide_layout)
    
    title = slide.shapes.title
    content = slide.placeholders[1]
    
    title.text = "이미지 설명 요약"
    content.text = result_text
    
    # 저장
    prs.save(output_filename)
    print(f"PPT 저장 완료: {output_filename}")

In [21]:
# PPT 생성
create_simple_ppt(retrieved_docs, result, "이미지_요약.pptx")

이미지 추가됨: ./images/refri7.png
이미지 추가됨: ./images/refri9.png
이미지 추가됨: ./images/refri4.png
이미지 추가됨: ./images/refri2.png
PPT 저장 완료: 이미지_요약.pptx
